# PyCity Schools Analysis

---

## District Summary:

In [1]:
# Import Dependecies
import pandas as pd
from pathlib import Path

In [2]:
# Set Path variables for OS indepence
schoolData_Path = Path("Resources/schools_complete.csv")
studentData_Path = Path("Resources/students_complete.csv")

# Read-in CSVs and convert to Pandas DataFrame objects
org_schoolDF = pd.read_csv(schoolData_Path)
org_studentDF = pd.read_csv(studentData_Path)

In [3]:
# Renaming Columns based off my own personal preference
org_schoolDF = org_schoolDF.rename(columns = {
    "school_name": "School Name",
    "type": "Type",
    "size": "Size",
    "budget": "Budget"})

org_studentDF = org_studentDF.rename(columns = {
    "student_name": "Student Name",
    "gender": "Gender",
    "grade": "Grade",
    "school_name": "School Name",
    "reading_score": "Reading Score",
    "math_score": "Math Score"})

In [ ]:
# Merging the two DataFrames by School name, using student DataFrame rows as keys
merge_stu_sch_DF = pd.merge(org_studentDF, org_schoolDF,
                                    how = "left", on = "School Name")
mainDF = merge_stu_sch_DF.copy()
mainDF

In [ ]:
# Calculate the total number of unique schools:
# ---------------------------------------------
# As usual, there are a lot of methods to calculate the number of unique schools & students
# 1)
#   totalSchools = mainDF["School Name"].nunique()
#   totalSchools
# 2)
#   totalSchools = len(org_schoolDF)
#   totalSchools

# My preferred method based off of current knowledge and desire to work with current DF
totalSchools = len(mainDF["School Name"].unique())
totalSchools

In [ ]:
# Calculate the total number of students:
# ---------------------------------------------
# 1)
#   totalStudents = mainDF["Student Name"].nunique()
#   totalStudents
# 2)
#   totalStudents = mainDF.iloc[-1,:]["Student ID"]
#   print(totalStudents + 1)
totalStudents = len(mainDF["Student ID"].unique())
totalStudents

In [ ]:
# # Calculate the total budget
totalBudget = org_schoolDF["Budget"].sum()
totalBudget

In [ ]:
# Calculate the average (mean) math score
avgMathScore = mainDF["Math Score"].mean()
avgMathScore

In [ ]:
# Calculate the average (mean) reading score
avgReadScore = mainDF["Reading Score"].mean()
avgReadScore

In [ ]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70):
#                         |Defaults to .loc method      |Assumes no NaNs, then just selects a specific cell (hierarchy inversion)
# percentPassMath = mainDF[(mainDF["Math Score"] >= 70)].count()["Student ID"]
# percentPassMath = percentPassMath / float(totalStudents) * 100
# print(percentPassMath)
# ---------------------------------------------------------------
# Method I procedurally wrote without looking at formula given:
percentPass_math = mainDF.loc[mainDF["Math Score"] >= 70, :]
percentPass_math = (len(percentPass_math) / totalStudents) * 100
percentPass_math

In [ ]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)
percentPass_read = mainDF[mainDF["Reading Score"] >= 70].count()["Student ID"]
percentPass_read = (percentPass_read / totalStudents) * 100
percentPass_read

In [ ]:
# Use the following to calculate the percentage of students that passed math and reading
percentPass_both = mainDF[(mainDF["Reading Score"] >= 70) & (mainDF["Math Score"] >= 70)].count()["Student ID"]
percentPass_both = (percentPass_both / totalStudents) * 100
percentPass_both

In [ ]:
# Delete previous mainDF, revert back to merge_stu_sch_DF for the complete merged DataFrame of students and schools
del mainDF

# Create a high-level snapshot of the district's key metrics in a DataFrame
districtSnap_DF = pd.DataFrame(
    [
        {
            "Total Schools": totalSchools,
            "Total Students": totalStudents,
            "Total Budget": totalBudget,
            "Average Math Score": avgMathScore,
            "Average Reading Score": avgReadScore,
            "% Passing Math": percentPass_math,
            "% Passing Reading": percentPass_read,
            "% Overall Passing": percentPass_both
        }
    ]
)
# Formatting
floatNumList = ["Average Math Score", "Average Reading Score"]
percentScoresList = ["% Passing Math", "% Passing Reading", "% Overall Passing"]

districtSnap_DF["Total Students"] = districtSnap_DF["Total Students"].map("{:,}".format)
districtSnap_DF["Total Budget"] = districtSnap_DF["Total Budget"].map("${:,.2f}".format)
districtSnap_DF[floatNumList] = districtSnap_DF[floatNumList].map("{:,.5f}".format)
districtSnap_DF[percentScoresList] = districtSnap_DF[percentScoresList].map("{:,.5f}".format)
districtSnap_DF = districtSnap_DF.set_index("Total Students")
del floatNumList, percentScoresList
## Display the DataFrame
display(districtSnap_DF)

## School Summary:

In [ ]:
# Use the code provided to select the type per school from school_data (original school DataFrame)
type_perSchool = org_schoolDF.set_index(["School Name"])["Type"]
type_perSchool

In [ ]:
# Calculate the total student count per school from school_data
size_perSchool = org_schoolDF.set_index(["School Name"])["Size"]
size_perSchool

In [16]:
# Calculate the total school budget and per capita spending per school from school_data
budget_perSchool = org_schoolDF.set_index("School Name")["Budget"]

capita_perSchool = budget_perSchool / size_perSchool
capita_perSchool = capita_perSchool.rename("Per Capita")

# <--- DELETE
# display(capita_perSchool)
# DELETE --->

In [17]:
# Calculate the average test scores per school from school_data_complete
avgMath_perSch = merge_stu_sch_DF.groupby("School Name")["Math Score"].mean()
avgRead_perSch = merge_stu_sch_DF.groupby("School Name")["Reading Score"].mean()

# <--- DELETE
# display(avgMath_perSch)
# display(avgRead_perSch)
# DELETE --->

In [ ]:
# Calculate the number of students per school with math scores of 70 or higher from school_data_complete
passMath_perSch = merge_stu_sch_DF.loc[merge_stu_sch_DF["Math Score"] >= 70, :]
passMath_perSch = passMath_perSch.groupby("School Name")["Math Score"].count()
passMath_perSch

In [ ]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
passRead_perSch = merge_stu_sch_DF.loc[merge_stu_sch_DF["Reading Score"] >= 70, :]
passRead_perSch = passRead_perSch.groupby("School Name")["Reading Score"].count()
passRead_perSch

In [ ]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
passBoth_perSch = merge_stu_sch_DF.loc[((merge_stu_sch_DF["Math Score"] >= 70) & (merge_stu_sch_DF["Reading Score"] >= 70)), :]
passBoth_perSch = passBoth_perSch.groupby("School Name")["Student ID"].count()
passBoth_perSch

In [21]:
# Use the provided code to calculate the passing rates
passMath_perSch_rate = passMath_perSch / size_perSchool * 100
passRead_perSch_rate = passRead_perSch / size_perSchool * 100
passBoth_perSch_rate = passBoth_perSch / size_perSchool * 100

In [ ]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.merge(type_perSchool, size_perSchool, how = "outer", on = "School Name")
per_school_summary = per_school_summary.join([budget_perSchool, capita_perSchool, avgMath_perSch, avgRead_perSch, 
                                            passMath_perSch_rate, passRead_perSch_rate, passBoth_perSch_rate], how = "outer")

# Formatting - Renaming and adding numerical symbols
per_school_summary = per_school_summary.rename(columns = {
    "Type": "School Type",
    "Size": "Total Students",
    "Budget": "Total School Budget",
    "Per Capita": "Per Student Budget",
    "Math Score": "Average Math Score",
    "Reading Score": "Average Reading Score",
    0: "% Passing Math",
    1: "% Passing Reading",
    2: "% Overall Passing"
})

floatCurrency_list = ["Total School Budget", "Per Student Budget"]
float_list = ["Average Math Score", "Average Reading Score", 
                "% Passing Math", "% Passing Reading", "% Overall Passing"]

per_school_summary[floatCurrency_list] = per_school_summary[floatCurrency_list].map("${:,.2f}".format)

# Display the DataFrame
display(per_school_summary)

## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.


## Bottom Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.


## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade

# Group by `school_name` and take the mean of the `math_score` column for each.

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`

# Minor data wrangling

# Display the DataFrame


## Reading Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade

# Group by `school_name` and take the mean of the the `reading_score` column for each.

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`

# Minor data wrangling

# Display the DataFrame


## Scores by School Spending

In [ ]:
# Establish the bins


In [ ]:
# Create a copy of the school summary for later aggregations


In [ ]:
# Use `pd.cut` on the per_school_capita Series from earlier to categorize per student spending based on the bins.

# Convert Spending Ranges (Per Student) to a string


In [ ]:
#  Calculate averages for the desired columns.


In [ ]:
# Assemble into DataFrame

# Display results


## Scores by School Size

In [ ]:
# Establish the bins.


In [ ]:
# Create a copy of the school summary for later aggregations


In [ ]:
# Use `pd.cut` on the per_school_counts Series from earlier to categorize school size based on the bins.

# Convert School Size to a string


In [ ]:
# Calculate averages for the desired columns.


In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`

# Display results


## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.


In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`

# Display results
